# Mortgage Lending Demo in Vertex AI + Snowflake

This notebook trains a simple **XGBoost** classifier on a mortgage-lending dataset, logs metrics and the model artifact, and registers the model in the **Snowflake Model Registry**—all from a Vertex AI Workbench instance.

## Prerequisites
* Upload **MORTGAGE_LENDING_DEMO_DATA.csv** to your Cloud Storage bucket, e.g.  
  `gs://mlops-xgb-bucket-1753032518/`
* Use a Vertex AI Workbench instance running **Python 3** (default kernel).  
* Make sure the Workbench VM’s service account has **Storage Object Viewer** (default for project editors) so the notebook can read from the bucket.

In [1]:
# Install the SDKs we need (run once, then restart the kernel)
!pip install --quiet \
    google-cloud-aiplatform \
    snowflake-ml-python \
    snowflake-connector-python \
    xgboost \
    pandas scikit-learn \
    gcsfs

In [2]:
!pip install --quiet toml

*After it finishes, click **Kernel ▸ Restart Kernel** so the new packages load.*

In [1]:
# Import necessary libraries
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

from snowflake.snowpark import Session
from snowflake.ml.registry import Registry

In [6]:
# ----------------------------
# Connect to your environment
# ----------------------------

# 1️⃣  Cloud Storage location of your CSV
GCS_URI = "gs://mlops-xgb-bucket-1753032518/MORTGAGE_LENDING_DEMO_DATA.csv"

# 2️⃣  Load Snowflake credentials from connections.toml
import toml
from pathlib import Path

TOML_PATH     = Path("connections.toml")   # adjust if stored elsewhere
PROFILE       = "connections"              # top-level section
SUB_PROFILE   = "Snowpark_MLOps_HOL"       # nested section

cfg_raw = toml.load(TOML_PATH)

# --- traverse the nested dict safely ---
try:
    section = cfg_raw[PROFILE][SUB_PROFILE]
except KeyError:
    raise KeyError(
        f"Section [{PROFILE}.{SUB_PROFILE}] not found in {TOML_PATH}"
    )

# --- build the Snowflake config dict ---
required_keys = ["account", "user", "role", "warehouse", "database", "schema"]
missing = [k for k in required_keys if k not in section]
if missing:
    raise ValueError(f"Missing key(s) in TOML section: {missing}")

SF_CFG = {
    "account":   section["account"],
    "user":      section["user"],
    "role":      section["role"],
    "warehouse": section["warehouse"],
    "database":  section["database"],
    "schema":    section["schema"],
}

# --- authentication: password vs. key-pair (JWT) ---
if "password" in section:
    SF_CFG["password"] = section["password"]

elif section.get("authenticator", "").lower() == "snowflake_jwt":
    # Expect a PEM file you already uploaded: rsa_private_key.pem
    from cryptography.hazmat.primitives import serialization

    KEY_PATH = Path("rsa_private_key.pem")   # adjust path if needed
    with open(KEY_PATH, "rb") as pem_in:
        private_key = serialization.load_pem_private_key(
            pem_in.read(),
            password=None,
        )
    SF_CFG["private_key"] = private_key

else:
    raise ValueError(
        "Neither 'password' nor 'authenticator = snowflake_jwt' provided."
    )

print("Loaded Snowflake connection for:",
      f"[{PROFILE}.{SUB_PROFILE}]")
print("GCS URI:", GCS_URI)

Loaded Snowflake connection for: [connections.Snowpark_MLOps_HOL]
GCS URI: gs://mlops-xgb-bucket-1753032518/MORTGAGE_LENDING_DEMO_DATA.csv


## Step 1: Read Model Training Data

In [7]:
# Read the same CSV straight from Cloud
# (GCS_URI was defined in your config cell)
import pandas as pd
df = pd.read_csv(GCS_URI)
print(f"Loaded {len(df):,} rows from GCS")
df.head()

Loaded 369,245 rows from GCS


,WEEK_START_DATE,WEEK,LOAN_ID,TS,LOAN_TYPE_NAME,LOAN_PURPOSE_NAME,APPLICANT_INCOME_000S,LOAN_AMOUNT_000S,COUNTY_NAME,MORTGAGERESPONSE
0,22-Dec-24,0,225846,51:21.6,VA-guaranteed,Refinancing,NaN,160,Erie County,1
1,22-Dec-24,0,298793,42:49.0,VA-guaranteed,Refinancing,109.0,255,Erie County,1
2,22-Dec-24,0,456295,29:48.5,Conventional,Home purchase,283.0,392,Westchester County,1
3,22-Dec-24,0,376334,55:14.9,FHA-insured,Refinancing,43.0,173,Albany County,0
4,22-Dec-24,0,216409,14:38.4,Conventional,Refinancing,209.0,255,Kings County,1


## Step 2: Prepare Model Features

In [11]:
# Drop rows missing target or any required feature
required_cols = [
    "MORTGAGERESPONSE",        # <-- This is your target variable
    "APPLICANT_INCOME_000S",
    "LOAN_AMOUNT_000S",
    "LOAN_TYPE_NAME",          # <-- This is a categorical column
    "LOAN_PURPOSE_NAME",       # <-- This is a categorical column
    "COUNTY_NAME",             # <-- This is a categorical column
]
df_clean = df.dropna(subset=required_cols)

# One-hot encode the categoricals (drop_first to avoid dummy trap)
categorical_cols = ["LOAN_TYPE_NAME", "LOAN_PURPOSE_NAME", "COUNTY_NAME"]
df_encoded = pd.get_dummies(
    df_clean,
    columns=categorical_cols,
    drop_first=True
)

# Split out X & y, keeping only numeric columns
X = df_encoded.drop(columns=["MORTGAGERESPONSE"])
X = X.select_dtypes(include=["number"]).copy()

# Drop any rows that still have NaNs (just in case)
X = X.dropna()

y = df_encoded.loc[X.index, "MORTGAGERESPONSE"] # <-- Match this to your target variable

## Step 3: Split into Train, Validation, and Holdout Sets (Split 80/20 then 75/25)

In [12]:
from sklearn.model_selection import train_test_split

# First split: 80% temp (train + val), 20% holdout (0.2 is your first test_size)
X_temp, X_holdout, y_temp, y_holdout = train_test_split(
    X, y, test_size=0.20, random_state=42, stratify=y
)
# Second split: 75% train, 25% val (0.25 is your second test_size)
X_train, X_val, y_train, y_val = train_test_split(
    X_temp, y_temp, test_size=0.25, random_state=42, stratify=y_temp
)

print(f"Train: {len(X_train)} | Val: {len(X_val)} | Holdout: {len(X_holdout)}")

Train: 189447 | Val: 63149 | Holdout: 63150


## Step 4: Train & Log XGBoost Model using 'logloss' as the evaluation metric in Vertex AI

In [15]:
from xgboost import XGBClassifier
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score
)

# 1️⃣  Train
model = XGBClassifier(eval_metric="logloss")
model.fit(X_train, y_train)

# 2️⃣  Validation predictions
val_preds = model.predict(X_val)

# 3️⃣  Compute metrics
val_acc  = accuracy_score(y_val, val_preds)
precision = precision_score(y_val, val_preds, average="macro")
recall    = recall_score(y_val, val_preds, average="macro")
f1        = f1_score(y_val, val_preds, average="macro")

# 4️⃣  Display results
print(f"Validation Accuracy: {val_acc:.4f}")
print(f"Precision (macro):   {precision:.4f}")
print(f"Recall   (macro):    {recall:.4f}")
print(f"F1       (macro):    {f1:.4f}")

Validation Accuracy: 0.9908
Precision (macro):   0.9889
Recall   (macro):    0.9851
F1       (macro):    0.9870


## Step 5: Evaluate Model Performance on Validation Set

In [16]:
# Step 5: Evaluate Model Performance on Validation Set (purely for display)
from sklearn.metrics import classification_report, confusion_matrix

# reuse val_preds from above
print("Classification Report:\n", classification_report(y_val, val_preds))
print("Confusion Matrix:\n", confusion_matrix(y_val, val_preds))

Classification Report:
               precision    recall  f1-score   support

           0       0.99      0.97      0.98     14590
           1       0.99      1.00      0.99     48559

    accuracy                           0.99     63149
   macro avg       0.99      0.99      0.99     63149
weighted avg       0.99      0.99      0.99     63149

Confusion Matrix:
 [[14219   371]
 [  211 48348]]


## Step 6: Allow Vertex AI to Access Snowflake
In order for your Vertex AI instance to connect to Snowflake, you must **add the current public IP address** of this notebook to your Snowflake network policy.

#### Step 6A: Get the Public IP of this Vertex AI Instance

In [17]:
!curl ifconfig.me

34.42.128.86

#### Step 6B: Add the IP to Your Snowflake Network Policy

Copy the Step 6A output (ex: `52.183.42.53`) and update your Snowflake network policy by running the following SQL in Snowsight (you must have `ACCOUNTADMIN` privileges):

## Step 7: Connect to Snowflake & Register the Model

In this step we will:

1. **Open a Snowpark session** using the connection details already loaded into **`SF_CFG`** from `connections.toml` (see the configuration cell near the top of the notebook).
2. **Verify the session context** by querying the current user, warehouse, database, and schema.
3. **Register our trained XGBoost model** in the **Snowflake Model Registry**, storing the model artifact plus evaluation metrics so it can be versioned, governed, and served for inference.

> **Security note**  
> All credentials are pulled from `connections.toml`, and the private key is read from `rsa_private_key.pem`, so no secrets are hard-coded inside the notebook.

In [20]:
from snowflake.snowpark import Session

# Open the session
session = Session.builder.configs(SF_CFG).create()

# Quick sanity check
session.sql("""
    SELECT current_user(), current_warehouse(),
           current_database(), current_schema()
""").show()

----------------------------------------------------------------------------------------
|"CURRENT_USER()"  |"CURRENT_WAREHOUSE()"  |"CURRENT_DATABASE()"  |"CURRENT_SCHEMA()"  |
----------------------------------------------------------------------------------------
|MLOPS_USER        |AICOLLEGE              |AICOLLEGE             |PUBLIC              |
----------------------------------------------------------------------------------------



## Step 8 — Register the Trained Model in Snowflake Model Registry  

### Step 8A: Create a Small Sample Input
We capture a few rows of the feature matrix (`X_train`) to help Snowflake infer the model’s input schema. A tiny sample keeps the registry lightweight.  

### Step 8B: Log the Model
Using the open **Snowpark session** (`session`) and the **Snowflake ML Registry** SDK:

1. Instantiate a `Registry` object tied to the current session.  
2. Call `log_model()` to upload the XGBoost model, sample input, and validation accuracy.  
3. The registry automatically versions the model (`v1`) under the name **`VERTEX_XGB_MORTGAGE`**.


In [22]:
from snowflake.ml.registry import Registry

# 8 A -- sample input data (5 rows of the features you trained on)
sample_input_data = X_train.head(5).copy()

# 8 B -- register the model
registry = Registry(session=session)

registry.log_model(
    model              = model,                   # trained XGBClassifier
    model_name         = "VERTEX_XGB_MORTGAGE",
    version_name       = "v1",
    sample_input_data  = sample_input_data,
    metrics            = {"accuracy": float(val_acc)},
)

print("✅   Model registered as 'VERTEX_XGB_MORTGAGE' version v1")

Logging model: creating model manifest...:  33%|███▎      | 2/6 [00:00<00:00,  5.16it/s]  

/opt/conda/lib/python3.10/site-packages/snowflake/ml/model/_model_composer/model_composer.py:231: UserWarning: `relax_version` is not set and therefore defaulted to True. Dependency version constraints relaxed from ==x.y.z to >=x.y, <(x+1). To use specific dependency versions for compatibility, reproducibility, etc., set `options={'relax_version': False}` when logging the model.
  self.manifest.save(


Model logged successfully.: 100%|██████████| 6/6 [01:41<00:00, 16.91s/it]                          
✅   Model registered as 'VERTEX_XGB_MORTGAGE' version v1


### Step 8 C — Create and Apply Governance Tags

Snowflake tags let you add metadata—such as deployment stage, business purpose, and lineage—to any database object, including models.  
Here we:

1. **Create the tag objects** (one-time DDL; run again if you need to update definitions).  
2. **Attach tag values** to the newly registered model version `VERTEX_XGB_MORTGAGE.v1`.  
3. **View the tags** to confirm they were applied.

In [25]:
# Create or replace governance tags (DDL is idempotent)
session.sql("CREATE OR REPLACE TAG MODEL_STAGE_TAG").collect()
session.sql("CREATE OR REPLACE TAG MODEL_PURPOSE_TAG").collect()
session.sql("CREATE OR REPLACE TAG SOURCE_TAG").collect()
session.sql("CREATE OR REPLACE TAG PROJECT_TAG").collect()

# Fetch the model object from the Registry
from snowflake.ml.registry import Registry
reg = Registry(session=session)
model_obj = reg.get_model("VERTEX_XGB_MORTGAGE")   # latest version (v1)

# Attach tag values
model_obj.set_tag("MODEL_STAGE_TAG",    "PROD")                       # deployment stage
model_obj.set_tag("MODEL_PURPOSE_TAG",  "Mortgage Response Scoring")  # business purpose
model_obj.set_tag("SOURCE_TAG",         "Vertex AI Notebook")         # origin
model_obj.set_tag("PROJECT_TAG",        "College of AI – MLOps HOL")  # traceability

# Display the tags to verify
model_obj.show_tags()

{'AICOLLEGE.PUBLIC.MODEL_PURPOSE_TAG': 'Mortgage Response Scoring',
 'AICOLLEGE.PUBLIC.MODEL_STAGE_TAG': 'PROD',
 'AICOLLEGE.PUBLIC.PROJECT_TAG': 'College of AI – MLOps HOL',
 'AICOLLEGE.PUBLIC.SOURCE_TAG': 'Vertex AI Notebook'}

In [26]:
# Apply tags and model-level metadata
m = reg.get_model("VERTEX_XGB_MORTGAGE")   # same name you used when registering

# Add model-level description
m.comment = "XGBoost classifier trained in Vertex AI to predict mortgage approval."

# Attach governance tags (these can overwrite or add to what you set earlier)
m.set_tag("MODEL_STAGE_TAG",   "PROD")                               # deployment stage
m.set_tag("MODEL_PURPOSE_TAG", "Mortgage Response Classification")   # business context
m.set_tag("SOURCE_TAG",        "Vertex AI Notebook")                 # origin
m.set_tag("PROJECT_TAG",       "College of AI – MLOps HOL")          # traceability

# View the tags you just set
m.show_tags()

{'AICOLLEGE.PUBLIC.MODEL_PURPOSE_TAG': 'Mortgage Response Classification',
 'AICOLLEGE.PUBLIC.MODEL_STAGE_TAG': 'PROD',
 'AICOLLEGE.PUBLIC.PROJECT_TAG': 'College of AI – MLOps HOL',
 'AICOLLEGE.PUBLIC.SOURCE_TAG': 'Vertex AI Notebook'}

## Step 9 — Pre-process Inference Data for Batch Scoring

The model was trained on:

* All numeric features in the mortgage-lending dataset  
* One-hot–encoded versions of **`LOAN_TYPE_NAME` , `LOAN_PURPOSE_NAME` , `COUNTY_NAME`**  
* Exactly the same column set (and order) as `X_train`

For batch scoring we must:

1. **Load the raw inference table** from Snowflake into a Pandas DataFrame.  
2. **Apply the same one-hot encoding** to the three categorical columns.  
3. **Add any dummy columns** that were present in `X_train` but are missing in the inference set (fill with `0`).  
4. **Re-order columns** to match `X_train`.  
5. **Run predictions** with the registered model in the Snowflake Model Registry.

In [29]:
# Pull inference data from Snowflake into Pandas
inference_df = (
    session
    .table("INFERENCEMORTGAGEDATA")      # adjust DB/Schema if needed
    .filter("WEEK = 1")                  # sample filter; remove if unnecessary
    .to_pandas()
)

# One-hot encode the same categorical columns
categorical_cols = ["LOAN_TYPE_NAME", "LOAN_PURPOSE_NAME", "COUNTY_NAME"]
inference_encoded = pd.get_dummies(
    inference_df,
    columns=categorical_cols,
    drop_first=True
)

# Add any missing dummy columns (present in X_train but absent here)
for col in X_train.columns:
    if col not in inference_encoded:
        inference_encoded[col] = 0

# Re-order columns to match the training feature order
inference_features = inference_encoded[X_train.columns]

## Step 10 — Batch-score Week 1 Applications and Persist Results

With the inference features prepared in **Step 9**, we can now:

1. **Run batch predictions** (`predict` and `predict_proba`) with the registered model version `VERTEX_XGB_MORTGAGE.v1`.
2. **Merge predictions back** onto the raw Week 1 input for observability.
3. **Write a unified results table** (`PREDICTIONS_WITH_GROUND_TRUTH`) to Snowflake so analysts can track model performance and drift over time.

This pattern separates **feature preparation** (Step 9) from **scoring + persistence** (this step), which mirrors the original workflow but now runs entirely inside Vertex AI Workbench + Snowflake.

In [34]:
import numpy as np
import pandas as pd
from IPython.display import display

reg   = Registry(session=session)
model = reg.get_model("VERTEX_XGB_MORTGAGE")  # container
mv     = model.version("v1")                  # or model.latest_version

# -------------------------------------------------------------
# Run inference with the registry model
# -------------------------------------------------------------
predictions       = mv.run(inference_features,  function_name="predict")
proba_predictions = mv.run(inference_features,  function_name="predict_proba")

# convert outputs to Series for easy concat
pred_series  = pd.Series(np.squeeze(predictions),                name="PREDICTION")
score_series = pd.Series(np.array(proba_predictions)[:, 1],      name="PREDICTED_SCORE")

# -------------------------------------------------------------
# Merge predictions with the raw Week-1 data
# -------------------------------------------------------------
results_df = inference_df.copy()
results_df["WEEK"]            = 1
results_df["PREDICTION"]      = pred_series
results_df["PREDICTED_SCORE"] = score_series

# -------------------------------------------------------------
# Persist the unified results back to Snowflake
# -------------------------------------------------------------
session.write_pandas(
    results_df,
    table_name="PREDICTIONS_WITH_GROUND_TRUTH",
    auto_create_table=True,   # create table if it doesn't exist
    overwrite=True            # replace Week-1 slice if you re-run
)

# Quick look at what we just wrote
display(
    session
    .table("PREDICTIONS_WITH_GROUND_TRUTH")
    .limit(10)
    .to_pandas()
)

,WEEK_START_DATE,WEEK,LOAN_ID,TS,LOAN_TYPE_NAME,LOAN_PURPOSE_NAME,APPLICANT_INCOME_000S,LOAN_AMOUNT_000S,COUNTY_NAME,MORTGAGERESPONSE,PREDICTION,PREDICTED_SCORE
0,1734652800000000000,1,361354,20:15.4,Conventional,Refinancing,59.8,293,Ulster County,0,0,0.000045
1,1734652800000000000,1,361354,20:15.4,Conventional,Refinancing,61.7,290,Ulster County,0,0,0.000041
2,1734652800000000000,1,361354,20:15.4,Conventional,Refinancing,61.8,304,Ulster County,0,0,0.000043
3,1734652800000000000,1,361354,20:15.4,Conventional,Refinancing,63.8,306,Ulster County,0,0,0.000032
4,1734652800000000000,1,361354,20:15.4,Conventional,Refinancing,59.0,281,Ulster County,0,0,0.000074
5,1734652800000000000,1,361354,20:15.4,Conventional,Refinancing,63.0,279,Ulster County,0,0,0.000054
6,1734652800000000000,1,361354,20:15.4,Conventional,Refinancing,53.0,306,Ulster County,0,0,0.000056
7,1734652800000000000,1,361354,20:15.4,Conventional,Refinancing,62.5,292,Ulster County,0,0,0.000041
8,1734652800000000000,1,361354,20:15.4,Conventional,Refinancing,60.2,305,Ulster County,0,0,0.000044
9,1734652800000000000,1,361354,20:15.4,Conventional,Refinancing,49.9,286,Ulster County,0,0,0.000062


## Step 11 — Full-Table Scoring (All Weeks) — Pandas-backed

Now that Week 1 scored correctly, let’s score **every row** in `INFERENCEMORTGAGEDATA` in a single Pandas workflow:

1. **Load the entire table** into a DataFrame with `session.table(...).to_pandas()`.  
2. **One-hot encode & align** the same three categoricals so the columns match `X_train`.  
3. **Fetch the registered model version** (`VERTEX_XGB_MORTGAGE.v1`).  
4. **Run `predict` and `predict_proba`** on the full DataFrame.  
5. **Write predictions back to Snowflake** in a unified results table.

> ⚠️ **Memory note**  
> Pulling millions of rows into a notebook can exhaust memory. For production-scale scoring, push steps 2–4 into Snowflake by encapsulating them in a VIEW or UDF and calling `mv.run()` on a Snowpark DataFrame instead of Pandas.

In [35]:
import pandas as pd
import numpy as np
from snowflake.ml.registry import Registry

# 1) Load the _entire_ inference dataset into pandas
raw_all = session.table("INFERENCEMORTGAGEDATA").to_pandas()

# 2) One-hot encode the same categoricals (drop_first to match training)
encoded_all = pd.get_dummies(
    raw_all,
    columns=categorical_cols,
    drop_first=True
)

# 3) Add any columns X_train had that this week didn’t, then reorder
for c in X_train.columns:
    if c not in encoded_all.columns:
        encoded_all[c] = 0
encoded_all = encoded_all[X_train.columns]

# 4) Load your model version from the registry
reg = Registry(session=session,
               database_name=conn_cfg["database"],
               schema_name=conn_cfg["schema"])
model = reg.get_model("VERTEX_XGB_MORTGAGE")    # <-- match your model_name

# 5) Run inference
preds  = mv.run(encoded_all,     function_name="predict")
probas = mv.run(encoded_all,     function_name="predict_proba")

# 6) Attach back into the raw dataframe
raw_all["PREDICTED_RESPONSE"] = np.squeeze(preds)
raw_all["PREDICTED_SCORE"]    = np.array(probas)[:,1]

# 7) Push it all back into Snowflake
results_sp = session.create_dataframe(raw_all)
results_sp.write.mode("overwrite") \
          .save_as_table("ALL_PREDICTIONS_WITH_GROUND_TRUTH")

# 8) Quick sanity-check
pdf = results_sp.limit(10).to_pandas()
display(pdf)

,WEEK_START_DATE,WEEK,LOAN_ID,TS,LOAN_TYPE_NAME,LOAN_PURPOSE_NAME,APPLICANT_INCOME_000S,LOAN_AMOUNT_000S,COUNTY_NAME,MORTGAGERESPONSE,PREDICTED_RESPONSE,PREDICTED_SCORE
0,2024-12-20,1,361354,20:15.4,Conventional,Refinancing,59.8,293,Ulster County,0,0,0.000045
1,2024-12-20,1,361354,20:15.4,Conventional,Refinancing,61.7,290,Ulster County,0,0,0.000041
2,2024-12-20,1,361354,20:15.4,Conventional,Refinancing,61.8,304,Ulster County,0,0,0.000043
3,2024-12-20,1,361354,20:15.4,Conventional,Refinancing,63.8,306,Ulster County,0,0,0.000032
4,2024-12-20,1,361354,20:15.4,Conventional,Refinancing,59.0,281,Ulster County,0,0,0.000074
5,2024-12-20,1,361354,20:15.4,Conventional,Refinancing,63.0,279,Ulster County,0,0,0.000054
6,2024-12-20,1,361354,20:15.4,Conventional,Refinancing,53.0,306,Ulster County,0,0,0.000056
7,2024-12-20,1,361354,20:15.4,Conventional,Refinancing,62.5,292,Ulster County,0,0,0.000041
8,2024-12-20,1,361354,20:15.4,Conventional,Refinancing,60.2,305,Ulster County,0,0,0.000044
9,2024-12-20,1,361354,20:15.4,Conventional,Refinancing,49.9,286,Ulster County,0,0,0.000062


## Scaling Inference with Snowflake ML Jobs and SPCS

Once you've validated your **Pandas-backed** batch scoring approach, production workloads typically require more scalable execution environments. Snowflake provides two powerful options:

### 📋 **Snowflake ML Jobs**
Schedule and orchestrate your end-to-end ML pipelines using Snowflake's managed **Container Runtime**.

**Key Benefits:**
-✅ **IDE Integration**: Dispatch jobs from VS Code, PyCharm, or Vertex AI Workbench
- ✅ **Serverless Execution**: Auto-scaling with no infrastructure management  
- ✅ **Cost-Effective**: Pay only for compute time used

**Best For:** Scheduled batch inference, model retraining pipelines, feature engineering workflows

---

### 🐳 **Snowpark Container Services (SPCS)**
Deploy your model as containerized services or jobs on dedicated compute pools with full flexibility.

**Key Benefits:**
- ✅ **No Package Restrictions**: Use any Python packages from PyPI
- ✅ **GPU Support**: Scale to large models with distributed GPU clusters
- ✅ **Service Endpoints**: Deploy always-on inference APIs

**Best For:** Real-time inference endpoints, large-scale bulk scoring, GPU-accelerated workloads

---

### 🎯 **Choosing the Right Approach**

| Scenario | Recommended Solution | Key Consideration |
|----------|---------------------|-------------------|
| **Scheduled batch inference** | ML Jobs | Finite-duration, cost-effective |
| **Always-on inference API** | SPCS Services | Set `MIN_INSTANCES = MAX_INSTANCES` |
| **Large bulk scoring** | SPCS Jobs | Handle enterprise-scale datasets |

### 📚 **Documentation & Next Steps**

- **[ML Jobs Guide](https://docs.snowflake.com/en/developer-guide/snowpark-ml/snowpark-ml-mlops)** - Automated ML pipelines
- **[SPCS Overview](https://docs.snowflake.com/en/developer-guide/snowpark-container-services/overview)** - Container services
- **[Model Serving](https://docs.snowflake.com/en/developer-guide/snowpark-ml/model-registry/model-serving-spcs)** - Deploy models to SPCS

**💡 Pro Tip:** Start with ML Jobs for your weekly batch scoring, then consider SPCS if you need real-time inference or GPU acceleration.

---